## Running the models

For understanding the effects of both plume and slab dynamics at phase transition, we will run the corresponding ASPECT parameter file as described in the previous section.

> In the cells below, it is assumed that the ASPECT directory is installed system-wide or that you are using the HUBzero ASPECT installation. If not, modify the ASPECT executable to the location where it is installed.

In [ ]:
%%capture
! aspect plume.prm

In [ ]:
%%capture
! aspect slab.prm

In [ ]:
# Load the relevant libraries
import pyvista as pv
import glob
from IPython.display import HTML

In [ ]:
def plot_deformation(output_dir, field, label, gif_file, cmap='RdBu'):
    '''
    Process a series of VTU files in the given output directory, returning the 'field'
    data for plotting.
    Inputs:
    -----------
    output_dir : str
        Path to the folder containing solution VTU files (expected in solution/solution-*.vtu)
    field : str
        Name of the field we want to look at in the solution
    label : str
        Label for the field being plotted
    '''

    solution_file_names = sorted(glob.glob(output_dir + 'solution/solution-*.pvtu'))

    mesh = pv.read(solution_file_names[0])

    plotter = pv.Plotter()

    # Create the first frame
    plotter.add_mesh(mesh, scalars=field, show_scalar_bar=False)
    plotter.add_axes()

    # We know that the model is XY plane so the following
    # lines just sets up the camera
    plotter.view_xy()
    plotter.camera.roll = 0
    plotter.camera.parallel_projection = True

    plotter.open_gif(gif_file, fps=4)

    # Update function for each frame
    def update(frame):
        frame_int = int(frame)
        new_mesh = pv.read(solution_file_names[frame_int])
        plotter.add_mesh(new_mesh, scalars=field, cmap='RdBu', \
                         scalar_bar_args={"vertical": True, "title": label})

         # Add a slice at y = 340 km (or 660 km from the top)
        slice_position = 340e3
        slice_mesh = mesh.slice(normal='y', origin=(0, slice_position, 0))
        plotter.add_mesh(slice_mesh, color='white', show_scalar_bar=False)

        plotter.add_text(f"Time step: {frame}", name="tstep")
        plotter.add_text(field,  position='upper_right', font_size=14)

    n_frames = len(solution_file_names)

    for i in range(n_frames):
        update(i)
        plotter.write_frame()

    plotter.close()

In [ ]:
# Make sure that the compositional field in the function
# matches what we have specified in the ASPECT input files:
# 'plume' and 'subducting_crust'
plot_deformation('plume/', 'plume', 'Composition', 'images/plume.gif')
plot_deformation('slab/', 'subducting_crust', 'Composition', 'images/slab.gif')

In [ ]:
# Now, you would have the gifs saved in the current directory
# this cell visualizes the model outputs side by side for
# comparison

HTML(f"""
<div style="display: flex; justify-content: center; align-items: center; gap: 10px;">
    <img src="{'images/plume.gif'}" width="750">
    <img src="{'images/slab.gif'}" width="750">
</div>
""")

In the cell above, you can see that both plume and slab stagnate, as expected, at around 660 km depth (marked by the white line). This is due to our chosen negative Clapeyron slope to represent the endothermic spinel–perovskite phase transition at this depth.

### Exercises

- Modify the Clapeyron slope to a positive value (e.g. 2 MPa/K in `Phase transition Clapeyron slopes`) and rerun the model. What changes do you observe in the plume and slab dynamics at 660 km depth?

- Modify the density contrast of the plume/slab with the surrounding mantle (`Density differential for compositional field 1`) and rerun the model. Do you still observe stagnation at 660 km depth? How does the density contrast affect the dynamics?

### Appendix:

After the model successfully runs, you should see a terminal output like below:

<div align="center">
<img src='./images/terminal_output.png' width="500"/>
</div>

The plume model takes approximately 12 seconds  while the slab model takes 1.5 minutes on 6 cores with 13th Gen Intel Core i7 processor.
Both models reach the linear and nonlinear solver tolerance.